In [25]:
!pip install openpyxl
!pip install pandas

import pandas as pd

# Load the COVID dataset
covid_dataset_path = '/content/drive/My Drive/Colab/covid_data_augmented.csv'
covid_data = pd.read_csv(covid_dataset_path)

# Load the Population dataset
population_dataset_path = '/content/drive/My Drive/Colab/co-est2023-pop.xlsx'
population_data = pd.read_excel(population_dataset_path)


,date,county_x,fips,cases,deaths,population,days_since_zero,cases_last_week,deaths_last_week,cases_per_100k,deaths_per_100k,risk_level,neighbor_population_sum
0,2020-03-24,Autauga,1001.0,1,0.0,58761.0,63,0.0,0.0,0.000000,0.0,Low,90517.0
1,2020-03-25,Autauga,1001.0,4,0.0,58761.0,64,0.0,0.0,0.000000,0.0,Low,90517.0
2,2020-03-26,Autauga,1001.0,6,0.0,58761.0,65,0.0,0.0,0.000000,0.0,Low,90517.0
3,2020-03-27,Autauga,1001.0,6,0.0,58761.0,66,0.0,0.0,0.000000,0.0,Low,90517.0
4,2020-03-28,Autauga,1001.0,6,0.0,58761.0,67,0.0,0.0,0.000000,0.0,Low,90517.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11599512,2022-05-09,Weston,56045.0,1589,18.0,6870.0,839,1.0,0.0,14.556041,0.0,Low,23185.0
11599513,2022-05-10,Weston,56045.0,1588,18.0,6870.0,840,-1.0,0.0,-14.556041,0.0,Low,23185.0
11599514,2022-05-11,Weston,56045.0,1588,18.0,6870.0,841,-1.0,0.0,-14.556041,0.0,Low,23185.0
11599515,2022-05-12,Weston,56045.0,1588,18.0,6870.0,842,-1.0,0.0,-14.556041,0.0,Low,23185.0


In [29]:
population_data[['county', 'state']]= population_data['Geographic Area'].str.split(' County, ', expand=True)
population_data["county"]=population_data["county"].str.lstrip(".")
target_states = ['California', 'Iowa', 'Florida', 'New York', 'Texas']
population_data = population_data[population_data['state'].isin(target_states)]
covid_data['date'] = pd.to_datetime(covid_data['date'])
covid_data['year'] = covid_data['date'].dt.year
covid_data_filtered = covid_data[covid_data['county_x'].isin(population_data['county'])]
covid_data_filtered['population'] = None
# Step 1: Precompute a Lookup Dictionary for Population Data
population_dict = {
    (row['county'], 2020): row['year2020'] for _, row in population_data.iterrows()
}
population_dict.update({
    (row['county'], 2021): row['year2021'] for _, row in population_data.iterrows()
})
population_dict.update({
    (row['county'], 2022): row['year2022'] for _, row in population_data.iterrows()
})

# Step 2: Create a Key Column in COVID Data for Lookup
covid_data_filtered['key'] = list(zip(covid_data_filtered['county_x'], covid_data_filtered['year']))

# Step 3: Map Population Data Using the Precomputed Dictionary
covid_data_filtered['population'] = covid_data_filtered['key'].map(population_dict)

# Step 4: Drop the Temporary Key Column (Optional)
covid_data_filtered = covid_data_filtered.drop(columns=['key'])

# Save the resulting dataset (if needed)
filtered_dataset_path = "/content/drive/My Drive/Colab/filtered_covid_data.csv"
covid_data_filtered.to_csv(filtered_dataset_path, index=False)

print(f"Filtered dataset saved successfully to {filtered_dataset_path}.")


<ipython-input-29-77fece56c2ea>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_data_filtered['population'] = None
<ipython-input-29-77fece56c2ea>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_data_filtered['key'] = list(zip(covid_data_filtered['county_x'], covid_data_filtered['year']))
<ipython-input-29-77fece56c2ea>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

Filtered dataset saved successfully to /content/drive/My Drive/Colab/filtered_covid_data.csv.
